In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

chrome_options = Options()
chrome_options.add_argument("--start-maximized")

service = Service(r"C:\Users\HADIL MARAI\Desktop\ODF\chromedriver-win64\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)

start_url = "https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-proposals?pageNumber=1&pageSize=50&sortBy=startDate&status=31094501,31094502"
driver.get(start_url)

with open("projets_EU_All.txt", "w", encoding="utf-8") as f:
    for page_num in range(1, 9):  # Pages 1 to 50
        try:
            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a.cft-call-list-table-title-link, a[href*='topic-details'], a[href*='competitive-calls-cs']"))
            )
        except Exception:
            print(f"❌ Aucun projet trouvé sur la page {page_num}.")
            break

        time.sleep(2)
        
        project_links = driver.find_elements(
            By.CSS_SELECTOR,
            "a.cft-call-list-table-title-link, a[href*='topic-details'], a[href*='competitive-calls-cs']"
        )

        print(f"📄 Page {page_num} : {len(project_links)} projets trouvés.")

        for link in project_links:
            titre = link.text.strip()
            href = link.get_attribute("href")

            try:
                container = link.find_element(By.XPATH, "./ancestor::eui-card")
            except:
                container = None

            # --- Main Status ---
            try:
                status_element = container.find_element(
                    By.CSS_SELECTOR, "span.eui-u-text-nowrap.eui-label"
                )
                status = status_element.text.strip()
            except:
                status = "Statut inconnu"

            # --- Dates and Additional Status ---
            opening_date = "Non spécifiée"
            next_deadline = "Non spécifiée"
            additional_status = "Non spécifié"

            try:
                result_card = container.find_element(
                    By.CSS_SELECTOR, "sedia-result-card-type.eui-u-display-block.eui-u-mt-xs.ng-star-inserted"
                )

                # Get all <strong> elements with date values
                strongs = result_card.find_elements(By.CSS_SELECTOR, "strong.ng-star-inserted")
                if len(strongs) >= 1:
                    opening_date = strongs[0].text.strip()
                if len(strongs) >= 2:
                    next_deadline = strongs[1].text.strip()

                # Get the last <span> that is not a "|" separator
                spans = result_card.find_elements(By.CSS_SELECTOR, "span.ng-star-inserted")
                for span in reversed(spans):
                    span_text = span.text.strip()
                    if span_text and span_text != "|":
                        additional_status = span_text
                        break

            except:
                pass

            # --- Output Results ---
            if titre and href and href.startswith("http"):
                f.write(
                    f"Titre: {titre}\n"
                    f"Lien: {href}\n"
                    f"Statut: {status}\n"
                    f"Date d'ouverture: {opening_date}\n"
                    f"Date limite: {next_deadline}\n"
                    
                )
                print(
                    f"✔ {titre} | {href} | Statut: {status} | "
                    f"📅 Ouverture: {opening_date} → Deadline: {next_deadline} | Extra: {additional_status}"
                )








        # Try clicking the next page icon
        try:
            next_icon = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'eui-icon-svg[aria-label="Go to next page"]'))
            )

            # Check if it's disabled
            is_disabled = next_icon.get_attribute("aria-disabled")
            if is_disabled == "true":
                print("✅ Fin de la pagination (bouton désactivé).")
                break

            # Scroll into view and click using JavaScript
            driver.execute_script("arguments[0].scrollIntoView(true);", next_icon)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", next_icon)

            print("➡️ Passage à la page suivante...")
            time.sleep(3)

            # Optional cache cleanup
            driver.delete_all_cookies()
            driver.execute_script("window.localStorage.clear();")
            driver.execute_script("window.sessionStorage.clear();")

        except Exception as e:
            print(f"⚠️ Erreur lors de la tentative de passage à la page suivante : {e}")
            break

driver.quit()


📄 Page 1 : 50 projets trouvés.
✔ MSCA Choose Europe for Science 2025 | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-MSCA-2025-COFUND-02-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate | Statut: Forthcoming | 📅 Ouverture: 01 October 2025 → Deadline: 03 December 2025 | Extra: Single-stage


✔ New CO2 capture technologies | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-24?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate | Statut: Forthcoming | 📅 Ouverture: 16 September 2025 → Deadline: 17 February 2026 | Extra: Single-stage
✔ Innovative pathways for low carbon and climate resilient building stock and built environment (Built4People Partnership) | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-03?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate | Statut: Forthcoming | 📅 Ouverture: 16 September 2025 → Deadline: 17 February 2026 | Extra: Single-stage
✔ Optimal combination of low embodied carbon construction products, technical building systems and circularity principles for climate neutral buildings (Built4People Partnership) | https://e

✔ Smarter buildings as part of the energy system for increased efficiency and flexibility – Societal Readiness Pilot | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-02?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate | Statut: Forthcoming | 📅 Ouverture: 16 September 2025 → Deadline: 17 February 2026 | Extra: Single-stage
✔ Competitiveness, energy security and integration aspects of advanced biofuels and renewable fuels of non-biological origin value chains | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-02?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate | Statut: Forthcoming | 📅 Ouverture: 16 September 2025 → Deadline: 17 February 2026 | Extra: Single-stage
✔ Innovative solutions for a generative AI-powered digital spine of the EU energy system 

✔ Phase out fossil fuel in energy intensive industries through the efficient integration of renewable energy sources | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-06?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate | Statut: Forthcoming | 📅 Ouverture: 16 September 2025 → Deadline: 17 February 2026 | Extra: Single-stage
✔ De-risking wave energy technology development through transnational pre-commercial procurement of wave energy research and development | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-13?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate | Statut: Forthcoming | 📅 Ouverture: 16 September 2025 → Deadline: 17 February 2026 | Extra: Single-stage
✔ On-site innovative robotic and automated solutions and techniques for more sustainable a

✔ Innovative approaches for the deployment of Positive Energy Districts | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-04?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate | Statut: Forthcoming | 📅 Ouverture: 16 September 2025 → Deadline: 17 February 2026 | Extra: Single-stage
✔ Towards commercialisation of Perovskite PV and development of dedicated manufacturing equipment (EUPI-PV Partnership) | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-10?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate | Statut: Forthcoming | 📅 Ouverture: 16 September 2025 → Deadline: 17 February 2026 | Extra: Single-stage
✔ Next generation distribution substation for increasing the system resilience | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/o

...

✔ Enable sustained coordination and guidance at the European level on institutional non-profit open access publishing | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-WIDERA-2025-06-ERA-02?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate | Statut: Open For Submission | 📅 Ouverture: 15 May 2025 → Deadline: 18 September 2025 | Extra: Single-stage
✔ Strengthening of the European Science for Policy Ecosystem | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-WIDERA-2025-06-ERA-06?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate | Statut: Open For Submission | 📅 Ouverture: 15 May 2025 → Deadline: 18 September 2025 | Extra: Single-stage
✔ Investigating and addressing career barriers faced by underrepresented and marginalised researchers | https://ec.europa.eu/info/funding-tenders/oppor

✔ Science comes to town 2027 | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-WIDERA-2025-06-ERA-07?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate | Statut: Open For Submission | 📅 Ouverture: 15 May 2025 → Deadline: 18 September 2025 | Extra: Single-stage
✔ Co-funded European partnership for Resilient Cultural Heritage | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL2-2025-03-HERITAGE-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate | Statut: Open For Submission | 📅 Ouverture: 15 May 2025 → Deadline: 16 September 2025 | Extra: Single-stage
✔ MSCA4Ukraine Fellowships 2025 | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-MSCA-2025-MSCA4UA-01-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pag

✔ Implementing co-funded action plans for connected regional innovation valleys in widening countries | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-WIDERA-2025-05-ACCESS-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate | Statut: Open For Submission | 📅 Ouverture: 14 May 2025 → Deadline: 15 October 2025 | Extra: Single-stage
✔ Skills for Tomorrow Training Programmes – Raw & Advanced Materials | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/competitive-calls-cs/10961?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate | Statut: Open For Submission | 📅 Ouverture: 13 May 2025 → Deadline: 29 August 2025 | Extra: Multiple Cut-off
✔ Game-changing innovations for European launch solutions | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/PPPA-2025-LAUNCH

✔ AIRISE Open Call 5 | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/competitive-calls-cs/10944?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate | Statut: Open For Submission | 📅 Ouverture: 08 May 2025 → Deadline: 16 July 2025 | Extra: Single-stage
✔ Survey preparation in the EU: Pest survey cards for 12 regulated pests | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/EUBA-EFSA-2025-PLANTS-04?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate | Statut: Open For Submission | 📅 Ouverture: 08 May 2025 → Deadline: 15 July 2025 | Extra: Single-stage
✔ European Excellence Initiative (EEI) | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-WIDERA-2025-01-ACCESS-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate 

➡️ Passage à la page suivante...


In [2]:
import pandas as pd

def parse_project_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f if line.strip()]

    data = []
    for i in range(0, len(lines), 5):
        data.append({
            "Title": lines[i].split(":", 1)[1].strip(),
            "Link": lines[i+1].split(":", 1)[1].strip(),
            "Status": lines[i+2].split(":", 1)[1].strip(),
            "Start_date": lines[i+3].split(":", 1)[1].strip(),
            "Deadline": lines[i+4].split(":", 1)[1].strip()
        })

    return pd.DataFrame(data)

df = parse_project_file("projets_EU_All.txt")
df.head()


,Title,Link,Status,Start_date,Deadline
0,MSCA Choose Europe for Science 2025,https://ec.europa.eu/info/funding-tenders/oppo...,Forthcoming,01 October 2025,03 December 2025
1,New CO2 capture technologies,https://ec.europa.eu/info/funding-tenders/oppo...,Forthcoming,16 September 2025,17 February 2026
2,Innovative pathways for low carbon and climate...,https://ec.europa.eu/info/funding-tenders/oppo...,Forthcoming,16 September 2025,17 February 2026
3,Optimal combination of low embodied carbon con...,https://ec.europa.eu/info/funding-tenders/oppo...,Forthcoming,16 September 2025,17 February 2026
4,Understand and minimise the environmental impa...,https://ec.europa.eu/info/funding-tenders/oppo...,Forthcoming,16 September 2025,17 February 2026


In [3]:
pd.set_option('display.max_colwidth', None)
print(df['Link'])

0        https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-MSCA-2025-COFUND-02-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate
1             https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-24?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate
2             https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-03?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate
3             https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-05?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate
4             https://ec.europa.eu/info/funding-tenders/opportun

In [4]:
df.head(15)

,Title,Link,Status,Start_date,Deadline
0,MSCA Choose Europe for Science 2025,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-MSCA-2025-COFUND-02-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Forthcoming,01 October 2025,03 December 2025
1,New CO2 capture technologies,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-24?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Forthcoming,16 September 2025,17 February 2026
2,Innovative pathways for low carbon and climate resilient building stock and built environment (Built4People Partnership),"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-03?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Forthcoming,16 September 2025,17 February 2026
3,"Optimal combination of low embodied carbon construction products, technical building systems and circularity principles for climate neutral buildings (Built4People Partnership)","https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-05?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Forthcoming,16 September 2025,17 February 2026
4,Understand and minimise the environmental impacts of offshore wind energy,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-08?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Forthcoming,16 September 2025,17 February 2026
5,Smarter buildings as part of the energy system for increased efficiency and flexibility – Societal Readiness Pilot,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-02?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Forthcoming,16 September 2025,17 February 2026
6,"Competitiveness, energy security and integration aspects of advanced biofuels and renewable fuels of non-biological origin value chains","https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-02?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Forthcoming,16 September 2025,17 February 2026
7,Innovative solutions for a generative AI-powered digital spine of the EU energy system,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-19?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Forthcoming,16 September 2025,17 February 2026
8,Innovative tools and services to manage and empower energy communities,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-20?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Forthcoming,16 September 2025,17 February 2026
9,Phase out fossil fuel in energy intensive industries through the efficient integration of renewable energy sources,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-06?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Forthcoming,16 September 2025,17 February 2026


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       373 non-null    object
 1   Link        373 non-null    object
 2   Status      373 non-null    object
 3   Start_date  373 non-null    object
 4   Deadline    373 non-null    object
dtypes: object(5)
memory usage: 14.7+ KB


In [6]:
df.shape

(373, 5)

### ODF DATASET 

In [7]:
import pandas as pd

# Données extraites
data = [
    {
        "Project Name": "FACTORIAT",
        "Description": "Support Deeptech & Hardware startups in prototyping and tech maturation with technical and financial help.",
        "Period": "2022–2023",
        "Axes / Thematic Areas": "Deeptech, Hardware, Incubation, Prototyping, Acceleration",
        "Region": "Tunisia",
        "Partners / Funders": "Industrial partners, ODF network",
        "Key Figures / Impact": "7 prototypes, 4 pre-industrial units"
    },
    {
        "Project Name": "National Entrepreneurship Program",
        "Description": "Design and implementation of Lesotho's national entrepreneurship ecosystem.",
        "Period": "2022–2025",
        "Axes / Thematic Areas": "Capacity building, Strategy, Startup Ecosystem",
        "Region": "Lesotho",
        "Partners / Funders": "Local government, ODF",
        "Key Figures / Impact": "500 entrepreneurs, 15 ESOs supported"
    },
    {
        "Project Name": "Arab Bank Strategy",
        "Description": "Develop funding & partnership strategy for African digital economy projects.",
        "Period": "N/A",
        "Axes / Thematic Areas": "Digital Economy, Strategic Development, Financing",
        "Region": "Sub-Saharan Africa",
        "Partners / Funders": "Arab Bank for Economic Development in Africa",
        "Key Figures / Impact": "Strategy developed"
    },
    {
        "Project Name": "World Bank Collaboration",
        "Description": "Develop startup ecosystem in 5 Southern African countries including Lesotho.",
        "Period": "2020–2022",
        "Axes / Thematic Areas": "Startup Ecosystem, Digital Economy, Entrepreneurship",
        "Region": "Southern Africa",
        "Partners / Funders": "World Bank",
        "Key Figures / Impact": "SA, Lesotho, Namibia, Botswana, Eswatini"
    },
    {
        "Project Name": "MDBAN – Business Angels Network",
        "Description": "Support early-stage Maghreb startups via diaspora angel investment.",
        "Period": "2021–Present",
        "Axes / Thematic Areas": "Startup Support, Investment, Diaspora Engagement",
        "Region": "MENA, Diaspora",
        "Partners / Funders": "MDBAN, ODF",
        "Key Figures / Impact": "56 startups financed, 33 angels"
    },
    {
        "Project Name": "BIATLABS",
        "Description": "Incubation program by BIAT Bank, run by ODF.",
        "Period": "2016–2018",
        "Axes / Thematic Areas": "Startup Incubation, Early-Stage Innovation",
        "Region": "Tunisia",
        "Partners / Funders": "BIAT (Private Bank)",
        "Key Figures / Impact": "4 cohorts, 45 startups, 15 labeled, 6 funded"
    },
    {
        "Project Name": "TECHNORIAT PPP Program",
        "Description": "Bridge research & entrepreneurship via incubation/acceleration of researchers.",
        "Period": "2021–2023",
        "Axes / Thematic Areas": "Scientific Research, Deeptech, Acceleration, Entrepreneurship",
        "Region": "Tunisia",
        "Partners / Funders": "TECHNORIAT, PPP",
        "Key Figures / Impact": "800 sensitized, 136 preselected, 13 incubated, 8 accelerated"
    },
    {
        "Project Name": "ABI – Applied Biotech & Innovation",
        "Description": "Turn biotech discoveries into marketable solutions with IP licensing model.",
        "Period": "N/A",
        "Axes / Thematic Areas": "Biotech, One Health, IP Licensing, Innovation",
        "Region": "N/A",
        "Partners / Funders": "ODF internal program",
        "Key Figures / Impact": "IP Model: License IN → Maturation → License OUT"
    }
]

# Création du DataFrame
odf_df = pd.DataFrame(data)
# Export en CSV
odf_df.to_csv("ODF_project_dataset.csv", index=False)


In [8]:
# Dictionnaire de correspondance EN => FR
all_keywords_text = {
    "strategic consulting": "conseil stratégique",
    "technical assistance": "assistance technique",
    "institutional support": "appui institutionnel",
    "tailored support": "accompagnement personnalisé",
    "innovation support": "accompagnement à l’innovation",
    "fundraising support": "accompagnement à la levée de fonds",
    "organizational development": "développement organisationnel",
    "capacity building": "développement de capacités",
    "program structuring": "structuration de programme",
    "project design": "ingénierie de projet",
    "project management": "gestion de projet",
    "program steering": "pilotage de programme",
    "growth strategy": "stratégie de croissance",
    "sector expertise": "expertise sectorielle",
    "project evaluation": "évaluation de projets",
    "due diligence": "due diligence",
    "project implementation": "mise en œuvre de projet",
    "roadmap development": "élaboration de feuille de route",
    "financial engineering": "ingénierie financière",
    "strategic diagnosis": "diagnostic stratégique",
    "monitoring and evaluation": "suivi-évaluation",
    "operational action plan": "plan d’action opérationnel",
    "public-private partnership": "partenariat public-privé",
    "ecosystem animation": "animation d’écosystème",
    "impact analysis": "analyse d’impact",
    "partnership facilitation": "facilitation de partenariats",
    "stakeholder mapping": "cartographie des acteurs",
    "knowledge transfer": "transfert de compétences",
    "collaborative innovation": "innovation collaborative",
    "open innovation": "open innovation",
    "economic development": "développement économique",
    "ecosystem development": "développement d’écosystèmes",
    "expansion strategy": "stratégie d’expansion",
    "development strategy": "stratégie de développement",
    "research valorization": "valorisation de la recherche",
    "technology transfer": "transfert de technologie",
    "technology maturation": "maturation technologique",
    "prototype industrialization": "industrialisation de prototypes",
    "technological co-development": "co-développement technologique",
    "intellectual property": "propriété intellectuelle",
    "research results": "résultats de recherche",
    "technology readiness level": "niveau de maturité technologique",
    "university spin-off": "spin-off universitaire",
    "research-industry collaboration": "collaboration recherche-industrie",
    "technology startup": "start-up technologique",
    "technology portfolio": "portefeuille technologique",
    "innovation commercialization": "commercialisation d’innovations",
    "proof of concept": "preuve de concept",
    "prototyping": "prototypage",
    "technology incubator": "incubateur technologique",
    "R&D support": "accompagnement à la R&D",
    "patent exploitation": "exploitation de brevets",
    "call for projects": "appel à projets",
    "call for proposals": "appel à propositions",
    "call for expression of interest": "appel à manifestation d’intérêt",
    "call for applications": "appel à candidatures",
    "call for consultants": "appel à consultants",
    "innovation grant": "subvention à l’innovation",
    "R&D project funding": "financement de projet R&D",
    "support program": "programme d’accompagnement",
    "support fund": "fonds de soutien",
    "innovation competition": "concours d’innovation",
    "incubation program": "programme d’incubation",
    "acceleration program": "programme d’accélération",
    "consulting assignment": "mission de conseil",
    "public procurement": "marché public",
    "strategic partnership": "partenariat stratégique",
    "service delivery": "prestation de services",
    "expert mission": "mission d’expertise",
    "deeptech": "deeptech",
    "biotechnology": "biotechnologie",
    "medtech": "medtech",
    "greentech": "greentech",
    "agritech": "agritech",
    "climatetech": "climatetech",
    "healthtech": "healthtech",
    "nanotechnology": "nanotechnologie",
    "robotics": "robotique",
    "artificial intelligence": "intelligence artificielle",
    "machine learning": "machine learning",
    "industry 4.0": "industrie 4.0",
    "advanced materials": "matériaux avancés",
    "clean technologies": "technologies propres",
    "renewable energy": "énergies renouvelables",
    "cybersecurity": "cybersécurité",
    "data science": "data science",
    "internet of things": "internet des objets",
    "sustainable development": "développement durable",
    "climate resilience": "résilience climatique",
    "economic inclusion": "inclusion économique",
    "empowerment through innovation": "autonomisation par l’innovation",
    "social innovation": "innovation sociale",
    "inclusive growth": "croissance inclusive",
    "social impact": "impact sociétal",
    "youth empowerment": "autonomisation des jeunes",
    "services digitalization": "digitalisation des services",
    "technological sovereignty": "souveraineté technologique",
    "territorial development": "développement territorial",
    "digital transformation": "transformation numérique",
    "researcher empowerment": "autonomisation des chercheurs",
    "diaspora integration": "intégration de la diaspora",
    "cross-sector collaboration": "collaboration intersectorielle",
    "industrial partnerships": "partenariats industriels",
    "market-driven innovation": "innovation orientée marché",
    "innovative SMEs": "PME innovantes",
    "industrial entrepreneurship": "entrepreneuriat industriel",
    "market access": "accès au marché",
    "value chain strengthening": "renforcement des chaînes de valeur",
    "economic impact": "impact économique",
    "economic empowerment": "autonomisation économique",
    "industrial competitiveness": "compétitivité industrielle",
    "applied research": "recherche appliquée",
    "skills development": "montée en compétences",
    "know-how transfer": "transfert de savoir-faire",
    "co-development": "co-développement",
    "technology leadership": "leadership technologique"
}


In [9]:
all_keywords_text

{'strategic consulting': 'conseil stratégique',
 'technical assistance': 'assistance technique',
 'institutional support': 'appui institutionnel',
 'tailored support': 'accompagnement personnalisé',
 'innovation support': 'accompagnement à l’innovation',
 'fundraising support': 'accompagnement à la levée de fonds',
 'organizational development': 'développement organisationnel',
 'capacity building': 'développement de capacités',
 'program structuring': 'structuration de programme',
 'project design': 'ingénierie de projet',
 'project management': 'gestion de projet',
 'program steering': 'pilotage de programme',
 'growth strategy': 'stratégie de croissance',
 'sector expertise': 'expertise sectorielle',
 'project evaluation': 'évaluation de projets',
 'due diligence': 'due diligence',
 'project implementation': 'mise en œuvre de projet',
 'roadmap development': 'élaboration de feuille de route',
 'financial engineering': 'ingénierie financière',
 'strategic diagnosis': 'diagnostic stra

### All the links

#### Detection de la pertinence par les mots clés sur tout les sites 

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

# === Configuration du navigateur Selenium (headless) ===
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

# === Fonction de nettoyage du texte ===
def clean_text(text):
    lines = text.split("\n")
    cleaned_lines = [line.strip() for line in lines if line.strip() and len(line.strip()) > 1]
    return "\n".join(cleaned_lines)

# === Fonction principale pour extraire et analyser une page ===
def analyze_url(url):
    try:
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(5)

        # Scroll pour charger le contenu dynamique
        for _ in range(10):
            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(0.3)
        time.sleep(3)

        # Extraction du texte
        js_code = """
        let desc = document.querySelector('div.showMore--three-lines');
        if (desc) {
            return desc.innerText.trim();
        } else {
            return 'Pas de description de projet détectée.';
        }
        """
        full_text = driver.execute_script(js_code)
        cleaned = clean_text(full_text)
        matched_keywords = [kw for kw in all_keywords_text if kw.lower() in cleaned.lower()]

        return {
            "URL": url,
            "Pertinence": "Yes" if matched_keywords else "No",
            "Matching Word(s)": ", ".join(matched_keywords) if matched_keywords else ""
        }

    except Exception as e:
        return {
            "URL": url,
            "Pertinence": "Error",
            "Matching Word(s)": str(e)
        }
    finally:
        driver.quit()

# === Charger tes liens depuis ton DataFrame ===
# Exemple :
# df_links = pd.read_csv("tes_liens.csv")  # ou déjà chargé
# ou si tu l'as déjà :
# df_links = df   # si df contient déjà la colonne 'link'

# Assure-toi d'avoir une colonne 'link' dans df
results = []
for url in df['Link']:
    print(f"Traitement : {url}")
    result = analyze_url(url)
    results.append(result)

# === Conversion en DataFrame final ===
df_final = pd.DataFrame(results)
df_final


Traitement : https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-MSCA-2025-COFUND-02-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate


Traitement : https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-24?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate


Traitement : https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-03?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate


Traitement : https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-05?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate


Traitement : https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-08?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate


...

Traitement : https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/competitive-calls-cs/10944?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate


Traitement : https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/EUBA-EFSA-2025-PLANTS-04?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate


Traitement : https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-WIDERA-2025-01-ACCESS-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate


Traitement : https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-MISS-2025-03-OCEAN-05?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate


,URL,Pertinence,Matching Word(s)
0,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-MSCA-2025-COFUND-02-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",No,
1,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-24?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",No,
2,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-03?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"climate resilience, social innovation"
3,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-05?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",No,
4,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-08?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"project design, renewable energy, economic impact"
...,...,...,...
368,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/competitive-calls-cs/10925?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate",No,
369,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/competitive-calls-cs/10944?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate",No,
370,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/EUBA-EFSA-2025-PLANTS-04?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate",No,
371,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-WIDERA-2025-01-ACCESS-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate",Yes,"capacity building, artificial intelligence, data science, digital transformation"


In [11]:
df_yes = df_final[df_final["Pertinence"] == "Yes"]

In [12]:
resultsYES_df_keywords = pd.DataFrame(df_yes)
resultsYES_df_keywords.to_excel("resultats_pertinenceYES_keywords.xlsx", index=False)
print("\n✅ Résultats sauvegardés dans ''.")


✅ Résultats sauvegardés dans ''.


#### Ajouter les colonnes de status et les dates

In [13]:
df_yes[['Title', 'Status', 'Start_date', 'Deadline']] = df[['Title', 'Status', 'Start_date', 'Deadline']]


C:\Users\HADIL MARAI\AppData\Local\Temp\ipykernel_41576\378187894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yes[['Title', 'Status', 'Start_date', 'Deadline']] = df[['Title', 'Status', 'Start_date', 'Deadline']]
C:\Users\HADIL MARAI\AppData\Local\Temp\ipykernel_41576\378187894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yes[['Title', 'Status', 'Start_date', 'Deadline']] = df[['Title', 'Status', 'Start_date', 'Deadline']]
C:\Users\HADIL MARAI\AppData\Local\Temp\ipykernel_41576\37818789

### Sortir seulement les projets qui sont Pertinents

In [14]:
df_yes_final = df_yes[df_yes['Pertinence'] == 'Yes'].copy()


In [15]:
df_yes_final

,URL,Pertinence,Matching Word(s),Title,Status,Start_date,Deadline
2,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-03?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"climate resilience, social innovation",Innovative pathways for low carbon and climate resilient building stock and built environment (Built4People Partnership),Forthcoming,16 September 2025,17 February 2026
4,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-08?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"project design, renewable energy, economic impact",Understand and minimise the environmental impacts of offshore wind energy,Forthcoming,16 September 2025,17 February 2026
5,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-02?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,renewable energy,Smarter buildings as part of the energy system for increased efficiency and flexibility – Societal Readiness Pilot,Forthcoming,16 September 2025,17 February 2026
6,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-02?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"sustainable development, industrial competitiveness","Competitiveness, energy security and integration aspects of advanced biofuels and renewable fuels of non-biological origin value chains",Forthcoming,16 September 2025,17 February 2026
13,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-04?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"artificial intelligence, renewable energy, social innovation",Innovative approaches for the deployment of Positive Energy Districts,Forthcoming,16 September 2025,17 February 2026
...,...,...,...,...,...,...,...
349,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL2-2025-01-DEMOCRACY-12?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate",Yes,"capacity building, research results",Community of democracy practitioners and researchers,Open For Submission,15 May 2025,16 September 2025
352,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL2-2025-01-DEMOCRACY-06?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate",Yes,knowledge transfer,Towards a European research hub on contemporary antisemitism and Jewish life and culture,Open For Submission,15 May 2025,16 September 2025
353,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-WIDERA-2025-06-ERA-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate",Yes,"innovation support, knowledge transfer, technology transfer, research results, skills development",Rolling out the ERA Policy Agenda results,Open For Submission,15 May 2025,18 September 2025
360,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL2-2025-03-HERITAGE-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate",Yes,"capacity building, knowledge transfer, call for proposals, sustainable development",Co-funded European partnership for Resilient Cultural Heritage,Open For Submission,15 May 2025,16 September 2025


##### LLM to chekc the pertinence of the projects by the projects of ODF

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
from groq import Groq
import re
from tqdm import tqdm

# === CONFIGURATION GROQ ===
client = Groq(api_key="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")
MODEL_NAME = "llama3-70b-8192"

# === CONFIG SELENIUM HEADLESS ===
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options)

# === TEXT CLEANUP & LIMITING FUNCTION ===
def clean_and_limit_text(text, max_words=800):
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    if len(words) > max_words:
        return ' '.join(words[:max_words]) + " [...] (texte tronqué)"
    return text

# === Extraction texte depuis URL ===
def extract_text_from_url(url):
    try:
        driver.get(url)
        time.sleep(5)

        for _ in range(10):
            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(0.3)
        time.sleep(5)

        js_code = """
        let desc = document.querySelector('div.showMore--three-lines');
        return desc ? desc.innerText.trim() : 'Pas de description de projet détectée.';
        """
        text = driver.execute_script(js_code)

        # Try also extracting UKRI accordion content
        try:
            open_all_btn = driver.find_element(By.CLASS_NAME, "govuk-accordion__open-all")
            open_all_btn.click()
            time.sleep(1)
        except:
            pass

        js_accordion = """
        let acc = document.querySelector('div.govuk-accordion.ukri-accordion');
        return acc ? acc.innerText.trim() : '';
        """
        accordion_text = driver.execute_script(js_accordion)

        full_text = f"{text}\n\n---\n\n{accordion_text}".strip()
        return full_text if full_text else "Pas de contenu détecté."

    except Exception as e:
        return f"Erreur Selenium: {e}"

# === Prompt pour Groq ===
def build_prompt(projects, site_text):
    projects_text = "\n".join([
        f"{i+1}. {p['Project Name']} : {p['Description']}, axe {p['Axes / Thematic Areas']}" 
        for i, p in enumerate(projects)
    ])
    return f"""
Tu es un expert en analyse de projets d'entreprise.

Voici une liste de projets que cette entreprise a déjà réalisés avec leurs descriptions et axes principaux :

{projects_text}

Voici maintenant le contenu d'un appel à projets spécifique extrait d'une page du portail européen :

{site_text}

Peux-tu me dire si cette opportunité est pertinente par rapport aux projets que l'entreprise a déjà réalisés ?  
Merci de répondre de façon claire :  
- Pertinence : Oui / Non  
- Projets similaires détectés : [liste]  
- Résumé rapide expliquant ta réponse.
"""

# === Requête à Groq ===
def query_groq(prompt, model=MODEL_NAME, max_completion_tokens=3000, temperature=1.0, stream=False, delay_between_requests=10):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Tu es un expert en comparaison de projets R&D."},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            max_completion_tokens=max_completion_tokens,
            top_p=1,
            stream=stream
        )

        if stream:
            result = ""
            for chunk in response:
                delta = chunk.choices[0].delta.content
                if delta:
                    result += delta
            time.sleep(delay_between_requests)
            return result.strip()
        else:
            time.sleep(delay_between_requests)
            return response.choices[0].message.content.strip()

    except Exception as e:
        return f"Erreur Groq: {e}"

# === Analyse du résultat Groq ===
def parse_result(text):
    pertinence = "Inconnu"
    resume = ""
    try:
        match = re.search(r"(?i)pertinence\s*[:\-–]\s*(oui|non)", text)
        if match:
            pertinence = match.group(1).capitalize()
        resume_match = re.search(r"(?i)résumé.*?:\s*(.+)", text)
        if resume_match:
            resume = resume_match.group(1).strip()
        else:
            resume = "\n".join(text.splitlines()[1:4]).strip()
    except:
        pass
    return pertinence, resume

# === Analyse d'une seule URL ===
def analyze_url(url, projects_list):
    print(f"🟡 Traitement : {url}")
    try:
        site_text_raw = extract_text_from_url(url)
        site_text = clean_and_limit_text(site_text_raw, max_words=800)
        projects_sample = projects_list[:3]  # Limiter à 3 projets
        prompt = build_prompt(projects_sample, site_text)
        result_text = query_groq(prompt)
        pertinence, resume = parse_result(result_text)

        return {
            "URL": url,
            "Pertinence LLM": pertinence,
            "Résumé LLM": resume,
            "Réponse brute": result_text
        }
    except Exception as e:
        return {
            "URL": url,
            "Pertinence LLM": "Erreur",
            "Résumé LLM": f"Erreur pendant l'analyse : {e}",
            "Réponse brute": str(e)
        }

# === MAIN ===
if __name__ == "__main__":
    try:
        projects_list = odf_df[['Project Name', 'Description', 'Axes / Thematic Areas']].dropna().to_dict(orient='records')
        url_list = df_yes['URL'].dropna().unique()
    except NameError:
        print("❌ Les DataFrames 'odf_df' et 'df_yes' doivent être définis avant l'exécution.")
        driver.quit()
        exit()

    results = []
    for url in tqdm(url_list, desc="🔍 Analyse des URLs"):
        result = analyze_url(url, projects_list)
        results.append(result)

    results_df = pd.DataFrame(results)
    df_final_llm = df_yes.merge(results_df, on="URL", how="left")

    df_final_llm.to_csv("df_yes_avec_pertinence_et_resume.csv", index=False)
    print("\n✅ Résultats enrichis sauvegardés dans 'df_yes_avec_pertinence_et_resume.csv'.")
    print(df_final_llm[['URL', 'Pertinence LLM', 'Résumé LLM']].head())

    driver.quit()


🔍 Analyse des URLs:   0%|                                                                                      | 0/88 [00:00<?, ?it/s]

🟡 Traitement : https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-03?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate


🔍 Analyse des URLs:   1%|▉                                                                             | 1/88 [00:39<56:36, 39.04s/it]

🟡 Traitement : https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-08?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate


🔍 Analyse des URLs:   2%|█▊                                                                            | 2/88 [01:04<44:08, 30.79s/it]

...

🔍 Analyse des URLs:  99%|████████████████████████████████████████████████████████████████████████████▏| 87/88 [37:24<00:27, 27.02s/it]

🟡 Traitement : https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-WIDERA-2025-01-ACCESS-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=8&pageSize=50&sortBy=startDate


🔍 Analyse des URLs: 100%|█████████████████████████████████████████████████████████████████████████████| 88/88 [37:51<00:00, 26.97s/it]

🔍 Analyse des URLs: 100%|█████████████████████████████████████████████████████████████████████████████| 88/88 [37:51<00:00, 25.81s/it]


✅ Résultats enrichis sauvegardés dans 'df_yes_avec_pertinence_et_resume.csv'.
                                                                                                                                                                                                                           URL  \
0  https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-03?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate   
1  https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-08?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate   
2  https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-02?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate   
3  https://ec.eur

In [17]:
df_final_llm.to_excel("df_LLM_ALL_EU.xlsx", index=False)
    

In [18]:

df_final_llm

,URL,Pertinence,Matching Word(s),Title,Status,Start_date,Deadline,Pertinence LLM,Résumé LLM,Réponse brute
0,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-03?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"climate resilience, social innovation",Innovative pathways for low carbon and climate resilient building stock and built environment (Built4People Partnership),Forthcoming,16 September 2025,17 February 2026,Non,"L'appel à projets est vide, il n'y a pas de description du projet, ce qui rend impossible la comparaison avec les projets précédents de l'entreprise. Il est donc difficile de déterminer si l'opportunité est pertinente ou non.","Pertinence : Non\n\nProjets similaires détectés : Aucun\n\nRésumé rapide : L'appel à projets est vide, il n'y a pas de description du projet, ce qui rend impossible la comparaison avec les projets précédents de l'entreprise. Il est donc difficile de déterminer si l'opportunité est pertinente ou non."
1,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-08?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"project design, renewable energy, economic impact",Understand and minimise the environmental impacts of offshore wind energy,Forthcoming,16 September 2025,17 February 2026,Non,**,"Bonjour !\n\nAprès avoir analysé l'appel à projets et les projets déjà réalisés par l'entreprise, voici ma réponse :\n\n**Pertinence : Non**\n\n**Projets similaires détectés : Aucun**\n\n**Résumé rapide :**\nL'appel à projets concerne spécifiquement la mise en œuvre de technologies pour réduire les impacts environnementaux de l'énergie éolienne offshore, ce qui est très loin des compétences et des domaines d'intervention de l'entreprise qui sont principalement liés à l'incubation, l'accélération et le développement de startups Deeptech, ainsi que la création d'écosystèmes d'entreprises. Il n'y a donc pas de similarité évidente entre cet appel à projets et les projets déjà réalisés par l'entreprise."
2,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D4-02?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,renewable energy,Smarter buildings as part of the energy system for increased efficiency and flexibility – Societal Readiness Pilot,Forthcoming,16 September 2025,17 February 2026,Non,"L'appel à projet concerne spécifiquement le domaine de la construction et de l'énergie (bâtiments intelligents, énergie renouvelable, etc.) qui n'est pas un axe d'expertise identifié dans les projets précédents de l'entreprise. Les projets précédents de l'entreprise sont plutôt orientés vers l'incubation, la stratégie, l'économie numérique, la formation, etc. Il n'y a aucunesimilarité évidente avec les projets déjà réalisés par l'entreprise.","Pertinence : Non\n\nProjets similaires détectés : Aucun\n\nRésumé rapide : L'appel à projet concerne spécifiquement le domaine de la construction et de l'énergie (bâtiments intelligents, énergie renouvelable, etc.) qui n'est pas un axe d'expertise identifié dans les projets précédents de l'entreprise. Les projets précédents de l'entreprise sont plutôt orientés vers l'incubation, la stratégie, l'économie numérique, la formation, etc. Il n'y a aucunesimilarité évidente avec les projets déjà réalisés par l'entreprise."
3,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-CL5-2026-02-D3-02?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"sustainable development, industrial competitiveness","Competitiveness, energy security and integration aspects of advanced biofuels and renewable fuels of non-biological origin value chains",For

In [19]:
df_final_llm.shape

(88, 10)

In [20]:
df_yes_final = df_final_llm[df_final_llm['Pertinence LLM'] == 'Oui'].copy()


In [21]:
df_yes_final.to_excel("df_LLM_YES_EU.xlsx", index=False)

In [22]:
df_yes_final

,URL,Pertinence,Matching Word(s),Title,Status,Start_date,Deadline,Pertinence LLM,Résumé LLM,Réponse brute
13,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-EIE-2026-01-CONNECT-03?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"economic impact, co-development",Scaling up deep tech ecosystems,Forthcoming,10 September 2025,20 January 2026,Oui,"L'appel à projets Europeen vise à soutenir le développement d'écosystèmes de deeptech autour d'infrastructures de recherche paneuropéennes, en générant des technologies à valeur marché et sociale. L'entreprise a déjà une expérience pertinente avec le projet FACTORIAT, qui soutient les startups deeptech et matérielle dans leur prototypage et maturité technique, ce qui correspond bien à l'objectif de cet appel à projets.","Pertinence : Oui\n\nProjets similaires détectés : FACTORIAT\n\nRésumé rapide : L'appel à projets Europeen vise à soutenir le développement d'écosystèmes de deeptech autour d'infrastructures de recherche paneuropéennes, en générant des technologies à valeur marché et sociale. L'entreprise a déjà une expérience pertinente avec le projet FACTORIAT, qui soutient les startups deeptech et matérielle dans leur prototypage et maturité technique, ce qui correspond bien à l'objectif de cet appel à projets."
14,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-EIE-2026-01-CONNECT-02?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"cybersecurity, digital transformation",Expanding Investment Ecosystems,Forthcoming,10 September 2025,20 January 2026,Oui,"Cette opportunité semble pertinente car elle vise à améliorer l'accès au financement et aux partenaires étrangers pour les start-ups issus de régions moins connectées et développées, ce qui est en ligne avec les objectifs de projets tels que le National Entrepreneurship Program qui vise à renforcer l'écosystème entrepreneurial national, et l'Arab Bank Strategy qui concerne le développement de la stratégie de financement et de partenariat pour les projets de l'économie numérique africaine. L'accent mis sur l'amélioration de l'accès au financement et aux partenaires étrangers, ainsi que la création de programmes d'accompagnement et de mise en réseau, sont des éléments clés qui lient cet appel à projets aux expériences précédentes de l'entreprise.","Pertinence : Oui\n\nProjets similaires détectés : National Entrepreneurship Program, Arab Bank Strategy\n\nRésumé rapide : Cette opportunité semble pertinente car elle vise à améliorer l'accès au financement et aux partenaires étrangers pour les start-ups issus de régions moins connectées et développées, ce qui est en ligne avec les objectifs de projets tels que le National Entrepreneurship Program qui vise à renforcer l'écosystème entrepreneurial national, et l'Arab Bank Strategy qui concerne le développement de la stratégie de financement et de partenariat pour les projets de l'économie numérique africaine. L'accent mis sur l'amélioration de l'accès au financement et aux partenaires étrangers, ainsi que la création de programmes d'accompagnement et de mise en réseau, sont des éléments clés qui lient cet appel à projets aux expériences précédentes de l'entreprise."
15,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-EIE-2026-01-CONNECT-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"capacity building, public procurement",European network of national competence centers for innovation procurement,Forthcoming,10 September 2025,20 January 2026,Oui,"Ce projet est pertinent pour l'entreprise en raison de son expérience dans la mise en place d'écosystèmes d'entrepreneuriat et de développement de stratégies de financement pour les entreprises innovantes, comme le montre 

In [23]:
df_yes_final.shape

(10, 10)

In [24]:
df_yes_final.to_csv("df_yes_final.csv", index=True)
print("\n✅ Résultats enrichis sauvegardés dans 'df_yes_final.csv'.")


✅ Résultats enrichis sauvegardés dans 'df_yes_final.csv'.


In [25]:
pd.set_option('display.max_colwidth', None)
print(df_yes_final['URL'])

13           https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-EIE-2026-01-CONNECT-03?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate
14           https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-EIE-2026-01-CONNECT-02?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate
15           https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-EIE-2026-01-CONNECT-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate
20                https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/DIGITAL-IRIS2-2025-QCI-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=2&pageSize=50&sortBy=startDate
28            https://ec.europa.eu/info/funding-

In [26]:
df_yes_final

,URL,Pertinence,Matching Word(s),Title,Status,Start_date,Deadline,Pertinence LLM,Résumé LLM,Réponse brute
13,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-EIE-2026-01-CONNECT-03?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"economic impact, co-development",Scaling up deep tech ecosystems,Forthcoming,10 September 2025,20 January 2026,Oui,"L'appel à projets Europeen vise à soutenir le développement d'écosystèmes de deeptech autour d'infrastructures de recherche paneuropéennes, en générant des technologies à valeur marché et sociale. L'entreprise a déjà une expérience pertinente avec le projet FACTORIAT, qui soutient les startups deeptech et matérielle dans leur prototypage et maturité technique, ce qui correspond bien à l'objectif de cet appel à projets.","Pertinence : Oui\n\nProjets similaires détectés : FACTORIAT\n\nRésumé rapide : L'appel à projets Europeen vise à soutenir le développement d'écosystèmes de deeptech autour d'infrastructures de recherche paneuropéennes, en générant des technologies à valeur marché et sociale. L'entreprise a déjà une expérience pertinente avec le projet FACTORIAT, qui soutient les startups deeptech et matérielle dans leur prototypage et maturité technique, ce qui correspond bien à l'objectif de cet appel à projets."
14,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-EIE-2026-01-CONNECT-02?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"cybersecurity, digital transformation",Expanding Investment Ecosystems,Forthcoming,10 September 2025,20 January 2026,Oui,"Cette opportunité semble pertinente car elle vise à améliorer l'accès au financement et aux partenaires étrangers pour les start-ups issus de régions moins connectées et développées, ce qui est en ligne avec les objectifs de projets tels que le National Entrepreneurship Program qui vise à renforcer l'écosystème entrepreneurial national, et l'Arab Bank Strategy qui concerne le développement de la stratégie de financement et de partenariat pour les projets de l'économie numérique africaine. L'accent mis sur l'amélioration de l'accès au financement et aux partenaires étrangers, ainsi que la création de programmes d'accompagnement et de mise en réseau, sont des éléments clés qui lient cet appel à projets aux expériences précédentes de l'entreprise.","Pertinence : Oui\n\nProjets similaires détectés : National Entrepreneurship Program, Arab Bank Strategy\n\nRésumé rapide : Cette opportunité semble pertinente car elle vise à améliorer l'accès au financement et aux partenaires étrangers pour les start-ups issus de régions moins connectées et développées, ce qui est en ligne avec les objectifs de projets tels que le National Entrepreneurship Program qui vise à renforcer l'écosystème entrepreneurial national, et l'Arab Bank Strategy qui concerne le développement de la stratégie de financement et de partenariat pour les projets de l'économie numérique africaine. L'accent mis sur l'amélioration de l'accès au financement et aux partenaires étrangers, ainsi que la création de programmes d'accompagnement et de mise en réseau, sont des éléments clés qui lient cet appel à projets aux expériences précédentes de l'entreprise."
15,"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-EIE-2026-01-CONNECT-01?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate",Yes,"capacity building, public procurement",European network of national competence centers for innovation procurement,Forthcoming,10 September 2025,20 January 2026,Oui,"Ce projet est pertinent pour l'entreprise en raison de son expérience dans la mise en place d'écosystèmes d'entrepreneuriat et de développement de stratégies de financement pour les entreprises innovantes, comme le montre 

In [27]:
df_yes_final.shape

(10, 10)

In [28]:
print(df_yes_final.columns.tolist())


['URL', 'Pertinence', 'Matching Word(s)', 'Title', 'Status', 'Start_date', 'Deadline', 'Pertinence LLM', 'Résumé LLM', 'Réponse brute']


In [29]:
df_yes_final.to_excel('df_final_yes.xlsx', index=False, engine='openpyxl')
